In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

# 데이터 전처리 파이프라인

In [4]:

from tqdm import tqdm
from datetime import datetime, timedelta
pd.set_option('mode.chained_assignment',  None)
class Processing:
    def __init__(self, x, test): # 이후 테스트 데이터도 넣는 버전 만들어야함
        self.x = x.set_index('ID')
        self.test = test
        
    
    def dummy(self):
        # 디버깅용 함수
        # airline, carrier id, state(출발,도착) 채워줌 
        # 캐리어 코드, 시간 채우는거 해야함
        data = self.fill_airline_and_id(self.x)
        data = self.fill_state(data)
        
        return data
    
    def time_padding(self, data):
        # 시간 패딩
        data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].fillna(0)
        data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(int)
        data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = data[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].astype(str)
        data['Estimated_Departure_Time'] = data['Estimated_Departure_Time'].str.pad(width=4, side='left', fillchar='0')
        data['Estimated_Arrival_Time'] = data['Estimated_Arrival_Time'].str.pad(width=4, side='left', fillchar='0')
        
        # 시간 둘 다 NULL값인 경우 제거함.
        # 제거하면 안될듯. 테스트데이터에도 있어서 어떻게든 해야함
        # 방안 1. 빈도수 높은거나 규칙에 따라 채운다.
        # 방안 2. 그냥 0으로 채우고 돌린다.
        data = data[~((data['Estimated_Departure_Time'] == '0000') & (data['Estimated_Arrival_Time'] == '0000'))]
        
        return data
    
    def fill_airline_and_id(self, data):
        '''
        Carrier ID - Airline 한 세트
        Carrier Code는 항공사, ID 겹침..
        '''
        
        
        
        
        
        
        
        for airline in data['Airline'].dropna().unique():

            id = data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'].dropna().unique()
            if id.shape[0] != 1:
                print(id)

            data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
                

        for id in data['Carrier_ID(DOT)'].dropna().unique():

            
            airline = data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'].dropna().unique()

            if airline.shape[0] != 1:
                print(airline)
            data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'] = airline[0]
                        
    
        return data
    
    def fill_state(self, data):
        for id in data['Origin_Airport_ID'].dropna().unique():
            data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'] = data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'].dropna().unique()[0]
        
        for id in data['Destination_Airport_ID'].dropna().unique():
            try:
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'].dropna().unique()[0]
            except: # 기록이 하나밖에 없음. Youngstown (YNG 공항)
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = 'Youngstown'
        return data
        

    def diff_time(self, df):
    
        time = timedelta(hours=df['H'], minutes=df['M']) - timedelta(hours=df['h'], minutes=df['m'])
        # print(time)
        time = round(time.total_seconds())
        
        hours = time // 3600
        minutes = (time % 3600) // 60
        
        if hours < 0:
            hours += 24
        
        # if minutes < 0:
        #     minutes += 60
        return hours, minutes 
        
    
    
    def make_noise_zero_data(self):
        # 시간 NULL값 없는 경우만 뽑음
        # nzdata = self.x[~(self.x(['Estimated_Departure_Time'] == '0000') | (self.x['Estimated_Arrival_Time'] == '0000'))]
        nzdata = self.x[~((self.x['Estimated_Departure_Time'] == '0000') | (self.x['Estimated_Arrival_Time'] == '0000'))]
        nzdata = self.make_time_feature(nzdata)
        time = nzdata.apply(self.diff_time, axis=1, result_type='expand')
        nzdata['diff_hour'] = time[0]
        nzdata['diff_minute'] = time[1]
        
        return nzdata
        

    
    def make_time_feature(self, data):
        data['h'] = [data['Estimated_Departure_Time'][i][:2] for i in range(len(data))]
        data['m'] = [data['Estimated_Departure_Time'][i][2:] for i in range(len(data))]
        data['H'] = [data['Estimated_Arrival_Time'][i][:2] for i in range(len(data))]
        data['M'] = [data['Estimated_Arrival_Time'][i][2:] for i in range(len(data))]

        data['h'] = data['h'].astype(int)
        data['m'] = data['m'].astype(int)
        data['H'] = data['H'].astype(int)
        data['M'] = data['M'].astype(int)

        return data
    
    def fill_time(self):
        print('시간 전처리')
        x = self.make_time_feature(self.x)
        check = self.make_noise_zero_data()
        
        # 출발시간
        print('출발시간')
        idx = x[x['Estimated_Departure_Time'] == '0000'].index
        for i in tqdm(idx):
            at = x.loc[i, 'Estimated_Arrival_Time']
            dt = x.loc[i, 'Estimated_Departure_Time']
            oa = x.loc[i, 'Origin_Airport']
            da = x.loc[i, 'Destination_Airport']
            al = x.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # print(check_data)
            
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Arrival_Time'] == at, 'Estimated_Departure_Time']
            if len(insert_time) != 0:
                try:
                    x.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index.drop('0000')[0]
                
                except:
                    x.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index[0]
                    
                x.loc[i, 'h'] = int(x.loc[i, 'Estimated_Departure_Time'][:2])
                x.loc[i, 'm'] = int(x.loc[i, 'Estimated_Departure_Time'][2:])
                
            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                

                try:
                    x.loc[i, 'Estimated_Departure_Time'] = check_data['Estimated_Departure_Time'].value_counts().index.drop('0000')[0]
                
                except:
                    print(at, dt, oa, da, al)
                    print(insert_time.value_counts())
                    print(insert_time.value_counts().index[0])
                    x.loc[i, 'Estimated_Departure_Time'] = check_data['Estimated_Departure_Time'].value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Departure_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Departure_Time'][2:])
            
            # 동일 시간 없을 시 평균 경과시간으로 대체
            # elif len(insert_time) == 0:
            #     ht = round(check_data['diff_hour'].mean())
            #     mt = round(check_data['diff_minute'].mean())
                
            #     insert_time =  timedelta(hours=x.loc[i, 'H'], minutes=x.loc[i, 'M']) - timedelta(hours=ht, minutes=mt)
            #     insert_time = round(insert_time.total_seconds())
            #     hours = insert_time // 3600
            #     minutes = (insert_time % 3600) // 60
                
            #     if hours < 0:
            #         hours += 24

            #     x.loc[i, 'h'] = hours
            #     x.loc[i, 'm'] = minutes
                
                
        # 도착시간
        print('도착시간')
        idx = x[x['Estimated_Arrival_Time'] == '0000'].index
        for i in tqdm(idx):
            at = x.loc[i, 'Estimated_Arrival_Time']
            dt = x.loc[i, 'Estimated_Departure_Time']
            oa = x.loc[i, 'Origin_Airport']
            da = x.loc[i, 'Destination_Airport']
            al = x.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Departure_Time'] == dt, 'Estimated_Arrival_Time']
            if len(insert_time) != 0:
                try:
                    x.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index.drop('0000')[0]
                
                except:
                    x.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Arrival_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Arrival_Time'][2:])
            
            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                try:
                    x.loc[i, 'Estimated_Arrival_Time'] = check_data['Estimated_Arrival_Time'].value_counts().index.drop('0000')[0]
                
                except: 
                    x.loc[i, 'Estimated_Arrival_Time'] = check_data['Estimated_Arrival_Time'].value_counts().index[0]
                    
                x.loc[i, 'H'] = int(x.loc[i, 'Estimated_Arrival_Time'][:2])
                x.loc[i, 'M'] = int(x.loc[i, 'Estimated_Arrival_Time'][2:])
            
            
            
            
            # 동일 시간 없을 시 평균 경과시간으로 대체
            # elif len(insert_time) == 0:
            #     ht = round(check_data['diff_hour'].mean())
            #     mt = round(check_data['diff_minute'].mean())
                
            #     # 23 50 / 1 40 -> 1 50
                
            #     insert_time =  timedelta(hours=x.loc[i, 'h'], minutes=x.loc[i, 'm']) + timedelta(hours=ht, minutes=mt)
            #     insert_time = round(insert_time.total_seconds())
                    
            #     hours = insert_time // 3600
            #     minutes = (insert_time % 3600) // 60
                
            #     if hours > 24:
            #         hours -= 24

            #     x.loc[i, 'h'] = hours
            #     x.loc[i, 'm'] = minutes
                
        return x
    
    
    def testset_processing(self):
        pass
    
    
                
                
            
        

In [5]:
pc = Processing(data, test)

ddata = pc.dummy()

In [161]:
ddata['Airline'].isna().sum()

11883

# dummy

In [7]:
'''
케리어 ID, 케리어 코드, 항공사 - 한 세트
도착 공항, 도착 항공 아이디, 지역 - 한 세트(출발 동일)
'''

# def fill_null_carrier(x):
    
    
data[['Airline', 'Carrier_Code(IATA)', 'Carrier_ID(DOT)']].value_counts().to_frame()

0
Airline                                   Carrier_Code(IATA) Carrier_ID(DOT)        
Southwest Airlines Co.                    WN                 19393.0          144837
Delta Air Lines Inc.                      DL                 19790.0           71282
United Air Lines Inc.                     UA                 19977.0           66749
American Airlines Inc.                    AA                 19805.0           65726
JetBlue Airways                           B6                 20409.0           32993
SkyWest Airlines Inc.                     UA                 20304.0           31285
                                          DL                 20304.0           24934
Alaska Airlines Inc.                      AS                 19930.0           23213
Envoy Air                                 AA                 20398.0           21835
Spirit Air Lines                          NK                 20416.0           19824
Comair Inc.                               AA                 20397.0           19701
Endeavor Air Inc.                         DL                 20363.0           18427
ExpressJet Airlines Inc.                  UA                 20366.0           14615
Frontier Airlines Inc.                    F9                 20436.0           13797
Republic Airlines                         UA                 20452.0           11972
Mesa Airlines Inc.                        UA                 20378.0           11512
Allegiant Air                             G4                 20368.0           10858
Air Wisconsin Airlines Corp               UA                 20046.0           10615
Horizon Air                               AS                 19687.0           10368
Republic Airlines                         AA                 20452.0            9935
Hawaiian Airlines Inc.                    HA                 19690.0            8672
SkyWest Airlines Inc.                     AA                 20304.0            8068
Mesa Airlines Inc.                        AA                 20378.0            8051
Capital Cargo International               AA                 20427.0            7852
Trans States Airlines                     UA                 20237.0            7057
Commutair Aka Champlain Enterprises, Inc. UA                 20445.0            5654
Republic Airlines                         DL                 20452.0            4697
SkyWest Airlines Inc.                     AS                 20304.0            4614
Compass Airlines                          DL                 21167.0            4528
GoJet Airlines, LLC d/b/a United Express  UA                 20500.0            4135
                                          DL                 20500.0            3496
Compass Airlines                          AA                 21167.0            2479
Virgin America                            VX                 21171.0            1053
Empire Airlines Inc.                      HA                 20263.0             844
ExpressJet Airlines Inc.                  DL                 20366.0             719
                                          AA                 20366.0             662
Trans States Airlines                     AA                 20237.0             330
Peninsula Airways Inc.                    AS                 20225.0             109
Cape Air                                  UA                 20253.0              96

In [162]:
df = data.__deepcopy__()

for airline in df['Airline'].dropna().unique():

    id = df.loc[df['Airline'] == airline, 'Carrier_ID(DOT)'].dropna().unique()


    df.loc[df['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
        

for id in df['Carrier_ID(DOT)'].dropna().unique():

    
    airline = df.loc[df['Carrier_ID(DOT)'] == id, 'Airline'].dropna().unique()

    df.loc[df['Carrier_ID(DOT)'] == id, 'Airline'] = airline[0]

In [224]:
df = data.__deepcopy__()
for id in df['Destination_Airport_ID'].dropna().unique():
    df.loc[df['Destination_Airport_ID'] == id, 'Destination_State'] = df.loc[df['Destination_Airport_ID'] == id, 'Destination_State'].dropna().unique()[0]
    # df.loc[df['Destination_Airport_ID'] == id, 'Destination_State'] = 1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [231]:
df[df['Destination_Airport_ID'] == 16133]


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
126925,TRAIN_126925,1,1,1500.0,1717.0,0,0,SFB,14761,Florida,YNG,16133,NaN,861.0,Allegiant Air,G4,20368.0,402NV,NaN


In [233]:
df.loc[df['Destination_Airport_ID'] == id, 'Destination_State']
id

df[df['Destination_Airport_ID'] == 16133]
df[(df['Distance'] == 861.0) & (df['Airline'] == 'Allegiant Air' )]
df[df['Destination_Airport'] == 'YNG']

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
126925,TRAIN_126925,1,1,1500.0,1717.0,0,0,SFB,14761,Florida,YNG,16133,NaN,861.0,Allegiant Air,G4,20368.0,402NV,NaN


In [240]:
ddata['Origin_State'].isna().sum()

0

In [241]:
df.groupby(['Carrier_Code(IATA)'])['Carrier_ID(DOT)'].nunique()

Carrier_Code(IATA)
AA    10
AS     4
B6     1
DL     7
F9     1
G4     1
HA     2
NK     1
UA    10
VX     1
WN     1
Name: Carrier_ID(DOT), dtype: int64

In [246]:
df = data.__deepcopy__()
for airline in df['Airline'].dropna().unique():

    id = df.loc[df['Airline'] == airline, 'Carrier_ID(DOT)'].dropna().unique()


    df.loc[df['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]

In [304]:
ddata['Origin_State'].isna().sum()

0

In [265]:
ddata[ddata['Carrier_ID(DOT)'].isna()]

ddata[['Origin_Airport_ID', 'Destination_Airport_ID', 'Carrier_Code(IATA)']].value_counts()
ddata.groupby(['Origin_Airport_ID', 'Destination_Airport_ID', 'Carrier_Code(IATA)', 'Distance', 'Tail_Number'])['Airline'].nunique()

Origin_Airport_ID  Destination_Airport_ID  Carrier_Code(IATA)  Distance  Tail_Number
10135              10397                   DL                  692.0     N132EV         1
                                                                         N153PQ         1
                                                                         N181PQ         1
                                                                         N195PQ         1
                                                                         N197PQ         1
                                                                                       ..
16869              13487                   DL                  553.0     N454SW         1
                                                                         N455SW         1
                                                                         N632SK         1
                                                                         N675BR         1
               

In [309]:
dum1 = ddata.drop(['Delay', 'Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1).dropna()
dum1= dum1.groupby(['Origin_Airport_ID', 'Destination_Airport_ID', 'Tail_Number'])['Airline'].nunique().reset_index()

print(dum1['Destination_Airport_ID'].unique().shape)
print(ddata['Destination_Airport_ID'].unique().shape)

(375,)
(375,)


In [310]:
dum2 = ddata.drop(['Delay', 'Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1).dropna()
dum2 = dum2.groupby(['Origin_Airport_ID', 'Destination_Airport_ID', 'Tail_Number'])['Carrier_ID(DOT)'].nunique().reset_index()

print(dum2['Destination_Airport_ID'].unique().shape)
print(ddata['Destination_Airport_ID'].unique().shape)

(375,)
(375,)


In [9]:
dum = ddata.drop(['Delay', 'Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1).dropna()
dum= dum.groupby(['Origin_Airport_ID', 'Destination_Airport_ID', 'Tail_Number'])['Airline'].nunique().reset_index()

dum['Airline'].value_counts()


1    584740
2       838
Name: Airline, dtype: int64

In [20]:
ddata[ddata['Airline'].isna()].set_index(['Origin_Airport_ID', 'Destination_Airport_ID', 'Tail_Number'])['Airline'].unique()

array([nan], dtype=object)

In [4]:
import ray
print(ray.__version__)
print(ray.init())
@ray.remote
def airline_test(df):
    count = 0
    dum = df[df['Airline'].isna()]
    for i in tqdm(range(len(dum))):
        oi = dum.iloc[i]['Origin_Airport_ID']
        di = dum.iloc[i]['Destination_Airport_ID']
        tn = dum.iloc[i]['Tail_Number']
        
        try:
            df.loc[((df['Origin_Airport_ID'] == oi) &
                (df['Destination_Airport_ID'] == di) &
                (df['Tail_Number'] == tn)), 'Airline'] = df.loc[((df['Origin_Airport_ID'] == oi) &
                (df['Destination_Airport_ID'] == di) &
                (df['Tail_Number'] == tn)), 'Airline'].dropna().unique()[0]
        
        except:
            count += 1
    
    print(count)
        
    return df


# put_data = ray.put(ddata)
# airline_test_data = ray.get(airline_test.remote(put_data))

2.2.0


2023-04-07 17:01:00,369	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.10', ray_version='2.2.0', ray_commit='b6af0887ee5f2e460202133791ad941a41f15beb', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:39491', 'raylet_socket_name': 'tcp://127.0.0.1:35218', 'webui_url': '127.0.0.1:8265', 'session_dir': 'C:\\Users\\SIGNLA~1\\AppData\\Local\\Temp\\ray\\session_2023-04-07_17-00-57_689038_25456', 'metrics_export_port': 21420, 'gcs_address': '127.0.0.1:34424', 'address': '127.0.0.1:34424', 'dashboard_agent_listen_port': 52365, 'node_id': '4410cb195489761c50190fc59449c343b519b8bfa84ef2d2987d012d'})


In [8]:
ddata[ddata['Airline'].isna()]

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
ID,,,,,,,,,,,,,,,,,,
TRAIN_000188,5,20,810.0,NaN,0,0,ORD,13930,Illinois,SEA,14747,Washington,1721.0,NaN,AS,NaN,N461AS,Not_Delayed
TRAIN_000388,3,17,1545.0,1829.0,0,0,MCO,13204,Florida,DTW,11433,Michigan,957.0,NaN,NaN,NaN,N509NK,NaN
TRAIN_000460,2,15,635.0,918.0,0,0,EVV,11612,Indiana,DTW,11433,Michigan,363.0,NaN,DL,NaN,N8886A,NaN
TRAIN_000622,9,7,522.0,644.0,0,0,GNV,11953,Florida,CLT,11057,North Carolina,388.0,NaN,NaN,NaN,N534EA,NaN
TRAIN_000664,1,24,1234.0,NaN,0,0,ATL,10397,Georgia,JAX,12451,Florida,270.0,NaN,DL,NaN,N918DH,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_999728,10,22,1751.0,2022.0,0,0,MSY,13495,Louisiana,ATL,10397,Georgia,425.0,NaN,DL,NaN,N678DL,Not_Delayed
TRAIN_999804,11,13,NaN,945.0,0,0,MSP,13487,Minnesota,DEN,11292,Colorado,680.0,NaN,WN,NaN,N8512U,NaN
TRAIN_999806,7,14,610.0,854.0,0,0,EWR,11618,New Jersey,TPA,15304,Florida,997.0,NaN,B6,NaN,N571JB,NaN


In [6]:
ddata['Carrier_Code(IATA)'].unique()

array(['WN', 'UA', 'AA', 'DL', nan, 'AS', 'B6', 'NK', 'F9', 'HA', 'G4',
       'VX'], dtype=object)

In [12]:
test_tn = test[(test['Airline'].isna())&(test['Carrier_ID(DOT)'].isna())]['Tail_Number'].unique()

test_tn.shape

(4845,)

In [13]:
ddata[(ddata['Airline'].isna())&(ddata['Carrier_ID(DOT)'].isna())]['Tail_Number'].unique().shape

(4880,)

# Tail Number - 항공사 정말 고유한가?

In [15]:
ddata[ddata['Airline'].isna()]

# ddata[['Carrier_Code(IATA)']]

ddata[ddata['Tail_Number'] == 'N461AS']

for t in ddata['Tail_Number'].unique():
    dum = ddata[ddata['Tail_Number'] == t]
    if len(dum['Airline'].dropna().unique()) != 1:
        print(t)
        
        print(dum['Airline'].unique())
        print('-'*50)   
        
        
    # if len(dum['Carrier_ID(DOT)'].dropna().unique()) != 1:
    #     print(t)
        
    #     print(dum['Carrier_ID(DOT)'].unique())
    #     print('-'*50)   
        
    # if len(dum['Carrier_Code(IATA)'].dropna().unique()) != 1:
    #     print(t)
        
    #     print(dum['Carrier_Code(IATA)'].unique())
    #     print('-'*50)   
        

N621VA
['Alaska Airlines Inc.' 'Virgin America' nan]
--------------------------------------------------
N531EG
['Envoy Air' 'Comair Inc.' nan]
--------------------------------------------------
N522AE
['Comair Inc.' 'Envoy Air' nan]
--------------------------------------------------
N523VA
['Alaska Airlines Inc.' 'Virgin America' nan]
--------------------------------------------------
N633VA
['Virgin America' 'Alaska Airlines Inc.' nan]
--------------------------------------------------
N656AE
['Capital Cargo International' 'Envoy Air']
--------------------------------------------------
N924VA
['Alaska Airlines Inc.' 'Virgin America']
--------------------------------------------------
N871AS
['ExpressJet Airlines Inc.' 'SkyWest Airlines Inc.' nan]
--------------------------------------------------
N153PQ
['Endeavor Air Inc.' 'ExpressJet Airlines Inc.']
--------------------------------------------------
N847VA
['Virgin America' 'Alaska Airlines Inc.' nan]
-------------------------------